We have learned to build micrograds and autograds from scratch. We used a scalar class called "Value". However, as a Neural Network grows in size, it is extremely computationally inefficient to work with scalars only. Thus we need to implement a tensor class which holds matrices of nxm dimensions.

In [37]:
import numpy as np
import itertools

In [38]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation


    def __repr__(self):
        return f"Tensor = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix - other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        out_matrix = other.matrix - self.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    
    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        out = tensor(out_matrix, (self, other), '*')
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        out = tensor(out_matrix, (self, other), '@')

        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        out = tensor(out_matrix, (self, ), 'T')

        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
        

In [39]:
test1 = [[3,5],
        [2,1]]
test2 = [[2,2],
        [2,3]]
t1 = tensor(test1)
t2 = tensor(test2)

t1 @ t2
t1.transpose()

Tensor = [[3 2]
 [5 1]]

In [40]:
a = tensor([[1, 2]])
b = tensor([[3, 4]])
c = a + b
d = c * a

print(d._prev)     
print(c._prev)     
print(d._operation) 
print(c._operation) 

{Tensor = [[4 6]], Tensor = [[1 2]]}
{Tensor = [[3 4]], Tensor = [[1 2]]}
*
+


Lets save our current state of Tensor class as a checkpoint. Similar to Value class for micrograd and autograd, we have implemented operations and children tracking. We have successfully implemented the computation graph model for tensors. We can now move on to back propagation.

In [41]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix - other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        out_matrix = other.matrix - self.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        out = tensor(out_matrix, (self, other), '*')
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        out = tensor(out_matrix, (self, other), '@')

        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        out = tensor(out_matrix, (self, ), 'T')

        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        

In [42]:
t = tensor()
print(t.grad)

None


Now let us add the backward lamda functions for each operation. This would be a headache. Either you can first derive them on paper yourself based on the concepts of Jacobians, downstream/upstream/local gradients, tensor contaction etc. or use standard formula. The first one is recommended for sound understanding. For reference, there are notes present in this directory.

In [43]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation
        self._backward = lambda : None
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    #Operations
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix

        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            self.grad += out.grad #Derivation in the notes. 
            other.grad += out.grad
        out = tensor(out_matrix, (self, other), '+')
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        return self + (-1 * other)
    
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        return other + (-1 * other)
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            self.grad += out.grad * other.matrix #Derivation in the notes. 
            other.grad += out.grad * self.matrix

        out = tensor(out_matrix, (self, other), '*')
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
    '''
    batch multiplication might cause shape broadcasts.
    eg. (3,2,2) @ (1,2,3) = (3,2,3)
    this is similar to our element wise operations
    thus we should be handling this the same way we did for elementwise operations
    But, for now, we would be working in a controlled way (Even for CNNS)
    and wouldn't need this handling.
    '''
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            self.grad += out.grad @ (other.matrix).swapaxes(-2,-1)#Derivation in the notes.
            other.grad += (self.matrix).swapaxes(-2,-1) @ out.grad 
        out = tensor(out_matrix, (self, other), '@')
        out._backward = _backward
        return out
    

    #I and thus we should learn at this point that to make our class compatible for ND tensors,
    #We need the matrix multiplication and Transpose backward to change
    #For higher dimensions, matmul = batch matmul where multiplication is done 
    #along each and every batches of 2D matrix. 
    #eg. If we have (2,3,3) shape tensor, it implies there are two batches of (3,3) matrices
    #similarly, (2,3,3,2) shape = 2x3 batches of 3x2 matrices.
    #matrix multiplication, (2,3,3) @ (2,3,2) = (2,3,2)
    def swap_axes(self, axis1, axis2):
        out_matrix = self.matrix.swapaxes(axis1, axis2)
        
        def _backward():
            self.grad = np.zeros_like(out.grad.swapaxes(axis1,axis2)) if self.grad is None else self.grad
            self.grad += (out.grad).swapaxes(axis1,axis2) #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out



    def transpose(self):
        out_matrix = self.matrix.transpose()
        
        def _backward():
            self.grad = np.zeros_like(out.grad.transpose()) if self.grad is None else self.grad
            self.grad += (out.grad).transpose() #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out
    
    def __pow__(self, N):
        assert isinstance(N, int | float), "Can only power up by scalars!"
        out_matrix = self.matrix ** N

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += N * (self.matrix ** (N-1)) * out.grad
        
        out = tensor(out_matrix, _children=(self, ), _operation="**")
        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        other = self.checkOther(other)
        return self * (other**-1)
    
    def __rtruediv__(self, other):
        return other * (self**-1)
    
    def sum(self):
        out_matrix = np.array(([[self.matrix.sum()]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad

        out = tensor(out_matrix, _children=(self, ), _operation='sum()')
        out._backward = _backward
        return out

    def mean(self):
        N = self.rows * self.columns
        out_matrix = np.array(([[self.matrix.sum()/(N)]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad / N

        out = tensor(out_matrix, _children=(self, ), _operation='mean()')
        out._backward = _backward
        return out
    
    def ReLU(self):
        out_matrix = np.maximum(0,self.matrix)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += (self.matrix > 0).astype(self.matrix.dtype) * out.grad

        out = tensor(out_matrix, (self, ), "ReLU")
        out._backward = _backward
        return out
    
    #Helper Functions
    def shape(self):
        return (self.rows, self.columns)
    
    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        
    def backward(self):
        self.grad = np.ones_like(self.matrix)
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for current in reversed(topo):

            current._backward()

In [44]:
a = tensor([[2.0, 3.0]])
b = tensor([[1.0, 4.0]])
c = a + b
d = c * a
d = d / 3
d

Tensor Values = [[2.00000006 7.00000021]]

In [45]:
Y_pred = tensor([[1.0, 2.0], [3.0, 4.0]])
Y_true = tensor([[1.5, 2.5], [2.5, 3.5]])

loss = ((Y_pred - Y_true) ** 2)
print("Predicted:", Y_pred)
print("True:", Y_true)
print("Loss:", loss.matrix.sum())

loss.backward()
print("Gradients for Y_pred:")
print(Y_pred.grad)
print("Gradients for Y_true:")
print(Y_true.grad)



Predicted: Tensor Values = [[1. 2.]
 [3. 4.]]
True: Tensor Values = [[1.5 2.5]
 [2.5 3.5]]
Loss: 1.0
Gradients for Y_pred:
[[-1. -1.]
 [ 1.  1.]]
Gradients for Y_true:
[[ 1.  1.]
 [-1. -1.]]


Now lets try to make it support all D tensors.

In [48]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        #assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        #self.rows = fromArray.shape[0]
        #self.columns = fromArray.shape[1]
        self.shape = fromArray.shape
        self._prev = set(_children)
        self._operation = _operation
        self._backward = lambda : None
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros(shape, dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    @classmethod
    def random(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(*shape)).astype(dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    @classmethod
    def const(cls, shape, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full(shape, constant)).astype(dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    #Operations
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix

        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            self.grad += out.grad #Derivation in the notes. 
            other.grad += out.grad
        out = tensor(out_matrix, (self, other), '+')
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        return self + (-1 * other)
    
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        return other + (-1 * other)
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            self.grad += out.grad * other.matrix #Derivation in the notes. 
            other.grad += out.grad * self.matrix

        out = tensor(out_matrix, (self, other), '*')
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other

    '''
    batch multiplication might cause shape broadcasts.
    eg. (3,2,2) @ (1,2,3) = (3,2,3)
    this is similar to our element wise operations
    thus we should be handling this the same way we did for elementwise operations
    But, for now, we would be working in a controlled way (Even for CNNS)
    and wouldn't need this handling.
    '''
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            self.grad += out.grad @ (other.matrix).swapaxes(-2,-1)#Derivation in the notes.
            other.grad += (self.matrix).swapaxes(-2,-1) @ out.grad 
        out = tensor(out_matrix, (self, other), '@')
        out._backward = _backward
        return out
    

    #I and thus we should learn at this point that to make our class compatible for ND tensors,
    #We need the matrix multiplication and Transpose backward to change
    #For higher dimensions, matmul = batch matmul where multiplication is done 
    #along each and every batches of 2D matrix. 
    #eg. If we have (2,3,3) shape tensor, it implies there are two batches of (3,3) matrices
    #similarly, (2,3,3,2) shape = 2x3 batches of 3x2 matrices.
    #matrix multiplication, (2,3,3) @ (2,3,2) = (2,3,2)
    def swap_axes(self, axis1, axis2):
        out_matrix = self.matrix.swapaxes(axis1, axis2)
        
        def _backward():
            self.grad = np.zeros_like(out.grad.swapaxes(axis1,axis2)) if self.grad is None else self.grad
            self.grad += (out.grad).swapaxes(axis1,axis2) #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out



    def transpose(self):
        out_matrix = self.matrix.transpose()
        
        def _backward():
            self.grad = np.zeros_like(out.grad.transpose()) if self.grad is None else self.grad
            self.grad += (out.grad).transpose() #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out
    
    def __pow__(self, N):
        assert isinstance(N, int | float), "Can only power up by scalars!"
        out_matrix = self.matrix ** N

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += N * (self.matrix ** (N-1)) * out.grad
        
        out = tensor(out_matrix, _children=(self, ), _operation="**")
        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        other = self.checkOther(other)
        return self * (other**-1)
    
    def __rtruediv__(self, other):
        return other * (self**-1)
    
    def sum(self):
        out_matrix = np.array(([[self.matrix.sum()]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad

        out = tensor(out_matrix, _children=(self, ), _operation='sum()')
        out._backward = _backward
        return out

    def mean(self):
        N = np.prod(self.shape)
        out_matrix = np.array(([[self.matrix.sum()/(N)]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad / N

        out = tensor(out_matrix, _children=(self, ), _operation='mean()')
        out._backward = _backward
        return out
    
    def ReLU(self):
        out_matrix = np.maximum(0,self.matrix)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += (self.matrix > 0).astype(self.matrix.dtype) * out.grad

        out = tensor(out_matrix, (self, ), "ReLU")
        out._backward = _backward
        return out
    
    #Helper Functions
    #def shape(self):
     #   return (self.rows, self.columns)
    
    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.shape, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape == self.shape, "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        
    def backward(self):
        self.grad = np.ones_like(self.matrix)
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for current in reversed(topo):

            current._backward()

In [49]:
shape = (2,2)
y = np.random.rand(*shape)
y

array([[0.20097739, 0.70269245],
       [0.58803901, 0.56874465]])

In [50]:
a = tensor([[[1,2],
            [2,3]]])
b = tensor([[1],
            [2]])
a@b

TypeError: 'tuple' object is not callable

In [58]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        #assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        #self.rows = fromArray.shape[0]
        #self.columns = fromArray.shape[1]
        self.shape = fromArray.shape
        self._prev = set(_children)
        self._operation = _operation
        self._backward = lambda : None
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros(shape, dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    @classmethod
    def random(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.randn(*shape) * 0.1).astype(dtype=dtype)
        t.shape = shape
        return t
    
    @classmethod
    def const(cls, shape, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full(shape, constant)).astype(dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    #Operations
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            out1 = self.return_unbroadcasted(out)
            out2 = other.return_unbroadcasted(out)
            self.grad += out1 #Derivation in the notes. 
            other.grad += out2
        out = tensor(out_matrix, (self, other), '+')
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        return self + (-1 * other)
    
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        return other + (-1 * other)
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            out1 = self.return_unbroadcasted(out)
            out2 = other.return_unbroadcasted(out)
            self.grad += out1* other.matrix #Derivation in the notes. 
            other.grad += out2 * self.matrix

        out = tensor(out_matrix, (self, other), '*')
        out._backward = _backward
        return out
    
    '''
    batch multiplication might cause shape broadcasts.
    eg. (3,2,2) @ (1,2,3) = (3,2,3)
    this is similar to our element wise operations
    thus we should be handling this the same way we did for elementwise operations
    But, for now, we would be working in a controlled way (Even for CNNS)
    and wouldn't need this handling.
    '''
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            self.grad += out.grad @ (other.matrix).swapaxes(-2,-1)#Derivation in the notes.
            other.grad += (self.matrix).swapaxes(-2,-1) @ out.grad 
        out = tensor(out_matrix, (self, other), '@')
        out._backward = _backward
        return out
    

    #I and thus we should learn at this point that to make our class compatible for ND tensors,
    #We need the matrix multiplication and Transpose backward to change
    #For higher dimensions, matmul = batch matmul where multiplication is done 
    #along each and every batches of 2D matrix. 
    #eg. If we have (2,3,3) shape tensor, it implies there are two batches of (3,3) matrices
    #similarly, (2,3,3,2) shape = 2x3 batches of 3x2 matrices.
    #matrix multiplication, (2,3,3) @ (2,3,2) = (2,3,2)
    def swap_axes(self, axis1, axis2):
        out_matrix = self.matrix.swapaxes(axis1, axis2)
        
        def _backward():
            self.grad = np.zeros_like(out.grad.swapaxes(axis1,axis2)) if self.grad is None else self.grad
            self.grad += (out.grad).swapaxes(axis1,axis2) #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out

    def transpose(self):
        out_matrix = self.matrix.transpose()
        
        def _backward():
            self.grad = np.zeros_like(out.grad.transpose()) if self.grad is None else self.grad
            self.grad += (out.grad).transpose() #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out
    
    def __rmatmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        return other @ self
    
    def __pow__(self, N):
        assert isinstance(N, int | float), "Can only power up by scalars!"
        out_matrix = self.matrix ** N

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            out1 = self.return_unbroadcasted(out)
            self.grad += N * (self.matrix ** (N-1)) * out1
        
        out = tensor(out_matrix, _children=(self, ), _operation="**")
        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        other = self.checkOther(other)
        return self * (other**-1)
    
    def __rtruediv__(self, other):
        return other * (self**-1)
    
    def sum(self):
        out_matrix = np.array(([[self.matrix.sum()]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad

        out = tensor(out_matrix, _children=(self, ), _operation='sum()')
        out._backward = _backward
        return out

    def mean(self):
        N = np.prod(self.shape)
        out_matrix = np.array(([[self.matrix.sum()/(N)]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad / N

        out = tensor(out_matrix, _children=(self, ), _operation='mean()')
        out._backward = _backward
        return out
    
    def ReLU(self):
        out_matrix = np.maximum(0,self.matrix)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += (self.matrix > 0).astype(self.matrix.dtype) * out.grad

        out = tensor(out_matrix, (self, ), "ReLU")
        out._backward = _backward
        return out
    
    def reshape(self, shape):
        assert isinstance(shape, tuple), f"Can only reshape using shape tuples e.g. (3,3). Provided is {shape}"
        out_matrix = self.matrix.reshape(shape)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += out.grad.reshape(self.shape)

        out = tensor(out_matrix, (self, ), "reshape()")
        out._backward = _backward
        return out
    
    def flatten(self):
        out_matrix = self.matrix.reshape(-1,np.prod(self.shape[1:]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += out.grad.reshape(self.shape)

        out = tensor(out_matrix, (self, ), "flatten()")
        out._backward = _backward
        return out
    
    #Helper Functions
    #def shape(self):
     #   return (self.rows, self.columns)

    def return_unbroadcasted(self, out):  
        added_axis = []
        stretched_axis = []
        for index, (first_no, second_no) in enumerate(itertools.zip_longest(reversed(self.shape), reversed(out.shape))):
            if first_no is None:
                added_axis.append(index)
            elif (first_no == 1) and (second_no > 1):
                stretched_axis.append(index)
        grad = out.grad
        ndim = len(out.shape)
        if stretched_axis:
            original_axes = tuple(ndim - 1 - i for i in stretched_axis)
            grad = np.sum(grad, axis=original_axes, keepdims=True)
        if added_axis:
            original_axes = tuple(ndim - 1 - i for i in added_axis)
            grad = np.sum(grad, axis=original_axes, keepdims=False)
        return grad

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.shape, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        #assert other.shape == self.shape, "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        
    def backward(self):
        self.grad = np.ones_like(self.matrix, dtype=float)
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for current in reversed(topo):

            current._backward()

    __array_ufunc__ = None

In [59]:
X = np.array([[11, 12, 13, 14],
             [21, 22, 23, 24]])

print("Before: ", X.shape)
print(X)
X = X.reshape(2,2,2)
print("After: ", X.shape)
print(X)
X = X.reshape(-1,*X.shape[1:])
print("After2: ", X.shape)
print(X)


Before:  (2, 4)
[[11 12 13 14]
 [21 22 23 24]]
After:  (2, 2, 2)
[[[11 12]
  [13 14]]

 [[21 22]
  [23 24]]]
After2:  (2, 2, 2)
[[[11 12]
  [13 14]]

 [[21 22]
  [23 24]]]


In [60]:
x = tensor.random((4,32,8,8))
y = x.flatten()
y.shape

(4, 2048)